In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### IMPORTS

In [ ]:
import matplotlib.pyplot as plt
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd 
import warnings; warnings.simplefilter('ignore')

In [ ]:
data = pd.read_csv("/kaggle/input/iris-dataset-uci/Iris.csv")
data.shape

In [ ]:
data.columns

In [ ]:
data.iloc[0:1,1:5]

In [ ]:
data.describe()

In [ ]:
sp = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
sp

In [ ]:
plt.scatter(data.SepalWidthCm,data.PetalWidthCm, c = [sp.index(val) for val in data.Species])

The iris flower classes are linearly separable

### TRAIN AND TEST DATA SPLIT

In [ ]:
train_x,test_x,train_y,test_y = train_test_split(data.iloc[:,1:5],data.iloc[:,5],test_size=0.2, random_state=101)
print("train x shape:", train_x.shape)
print("train y shape:", train_y.shape)
print("test x shape:", test_x.shape)
print("test y shape:", test_y.shape)

In [ ]:
print(train_x.columns)

### SOFT MARGIN SVM

In [ ]:
def svm_prediction(c_value):
    svm_obj = LinearSVC(C = c_value, loss="hinge", multi_class="crammer_singer")
    svm_obj.fit(train_x,train_y)
    y_pred = svm_obj.predict(test_x)
    test_err = np.sum(y_pred != test_y) / len(test_y)
    return test_err

In [ ]:
for c in [0.01,0.1,1.0,10,100]:
    test_err = svm_prediction(c)
    print(f"Error rate : {test_err} for c : {c}")

In [ ]:
# Linear SVM with C value = 1.0

svm_obj = LinearSVC(C = 1.0, loss="hinge", multi_class="crammer_singer")
svm_obj.fit(train_x,train_y)
y_pred = svm_obj.predict(test_x)
print("Accuracy score Linear SVM:",accuracy_score(test_y,y_pred)*100)

In [ ]:
#coefficient : w
w = svm_obj.coef_
print("w: ", w )
#intercept : b
b = svm_obj.intercept_
print("b: ", b)

#### SVM hyperplane for each class   

$ w_1.x + b_1$   
Class 1   
$ 0.37.SepalLength + 1.08.SepalWidth -1.22.PetalLength -0.84.PetalWidth + 0.2$  
Class 2  
$ 0.34.SepalLength + 0.08.SepalWidth -0.25.PetalLength -0.76.PetalWidth + 0.9$  
Class 3  
$ -0.72.SepalLength - 1.17.SepalWidth +1.47.PetalLength +1.6.PetalWidth - 1.12$

In [ ]:
# Polynomial SVM

svm_obj2 = SVC(C=1.0,kernel="poly", degree = 3)
svm_obj2.fit(train_x,train_y)
y_pred2 = svm_obj2.predict(test_x)
print("Accuracy score - Polynomial SVM:", accuracy_score(test_y,y_pred2)*100)

### VISUALISATION

##### SVM WITH 2 FEATURES

In [ ]:
#Features selected-Petal width and Sepal width
x = np.array(train_x.iloc[:,[1,3]]) 
y = np.array(train_y)
print(train_x.iloc[:,[1,3]].columns)

In [ ]:
def display_data_and_boundary(x,y,pred_fn):
    x1min = min(x[:,0]) - 1
    x1max = max(x[:,0]) + 1
    x2min = min(x[:,1]) - 1
    x2max = max(x[:,1]) + 1
    plt.xlim(x1min,x1max)
    plt.ylim(x2min,x2max)
    
    k = int(len(np.unique(y)))
    cols = ["ro","k^","b*"]
    
    #plotting the data points for each labels
    for label in range(k):
        plt.plot(x[y==sp[label],0],x[y==sp[label],1],cols[label] , markersize = 8)  
    
    xx = np.arange(x1min, x1max, 0.05)
    yy = np.arange(x2min, x2max, 0.05)
    xx, yy = np.meshgrid(xx,yy)
    grid = np.c_[xx.ravel(),yy.ravel()]
    Z = np.array([sp.index(pred_fn(point)) for point in grid])
    
    #Show the classifier boundary using a color plot
    Z = Z.reshape(xx.shape)
    plt.pcolormesh(xx,yy,Z, cmap= plt.cm.Pastel1, vmin=0, vmax=k)
    plt.show()

In [ ]:
svm_obj3 = LinearSVC(loss="hinge", multi_class="crammer_singer", C=1.0)
svm_obj3.fit(x,y)
pred_fn = lambda p : svm_obj3.predict(p.reshape(1,-1))

display_data_and_boundary(x,y,pred_fn)

In [ ]:
#coefficient : w
w = svm_obj3.coef_
print("w: ", w )
#intercept : b
b = svm_obj3.intercept_
print("b: ", b)

In [ ]:
decision_fn = svm_obj.decision_function(x)
decision_fn[0]

# decision function returns probability of that point to be seen under each Iris species (each class)
# decision_function = np.dot(X, clf.coef_[0]) + clf.intercept_[0])

In [ ]:
print(np.dot(x[0],w[0,:])+b[0], end="\t")
print(np.dot(x[0],w[1,:])+b[1], end="\t")
print(np.dot(x[0],w[2,:])+b[2], end="\t")

In [ ]:
svm_obj.predict(x[0:1]) # the model predicts the species which has highest probability

##### DISPLAY DATA, BOUNDARY WITH SUPPORT VECTOR

In [ ]:
def display_boundary_supportvectors(x,y,pred_fn,sv):
    x1min = min(x[:,0]) - 1
    x1max = max(x[:,0]) + 1
    x2min = min(x[:,1]) - 1
    x2max = max(x[:,1]) + 1
    plt.xlim(x1min,x1max)
    plt.ylim(x2min,x2max)
    
    k = int(len(np.unique(y)))
    cols = ["ro","k^","b*"]
    
    #plotting the data points for each labels
    for label in range(k):
        plt.plot(x[y==sp[label],0],x[y==sp[label],1],cols[label] , markersize = 8)  
    
    xx = np.arange(x1min, x1max, 0.05)
    yy = np.arange(x2min, x2max, 0.05)
    xx, yy = np.meshgrid(xx,yy)
    grid = np.c_[xx.ravel(),yy.ravel()]
    Z = np.array([sp.index(pred_fn(point)) for point in grid])
    
    #Show the classifier boundary using a color plot
    Z = Z.reshape(xx.shape)
    plt.pcolormesh(xx,yy,Z, cmap= plt.cm.Pastel1, vmin=0, vmax=k)
    plt.scatter(support_vectors[:,0],support_vectors[:,1],s=100,facecolors="none",edgecolors="k",linewidth=1)
    plt.show()

In [ ]:
support_vector_indices = np.where(np.abs(decision_fn) <= 1+1e-15)[0]
support_vectors = x[support_vector_indices]
display_boundary_supportvectors(x,y,pred_fn,support_vectors)